In [1]:
#Python code for scraping Economist website and fetching Business school ranking data 
#using Beautiful Soup
import urllib2
import re, requests
from bs4 import BeautifulSoup
opener = urllib2.build_opener()
opener.addheaders = [('User-agent', 'Chrome/52.0')]
university = []
#Extract the first level of information about the university B'schools, ranks and their locations
for i in range(10):
    if (i == 0):
        url = "http://www.economist.com/whichmba/full-time-mba-ranking"
    else:
        url = "http://www.economist.com/whichmba/full-time-mba-ranking?page=" + str(i)
    response = opener.open(url)
    page = response.read()
    #print i
    university.append(BeautifulSoup(page))


C:\Users\trichna\AppData\Local\Continuum\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [153]:
#Extract the details regarding the university Business school name, its rank and location
#First extract all the tags
univRankList = []
univSchoolList = []
univLocationList = []
for i in range(10):
    univrank = university[i].find_all('td', {'class': "views-field views-field-field-wmba-school-rank-overall-value active"})
    univRankList.append(univrank)
    univSchool = university[i].find_all('td', {'class': "views-field views-field-field-wmba-school-name-alpha-value"})
    univSchoolList.append(univSchool)
    univLocation = university[i].find_all('td', {'class': "views-field views-field-name"})
    univLocationList.append(univLocation)
    
univ_school = []; univ_rank =[]; univ_location = []; univ_school_url = []

#Once the tags are extracted, fetch the value contained within those tags
for i in range(10):
    univ_school.append([tag.get_text().strip().encode('utf-8') for tag in univSchoolList[i]])
    univ_rank.append([tag.get_text().strip().encode('utf-8') for tag in univRankList[i]])
    univ_location.append([tag.get_text().strip().encode('utf-8') for tag in univLocationList[i]]) 
    for j in range(10):
        univ_school_url.append(univSchoolList[i][j].a.get('href'))

# Flatten the list of lists by performing  an aggregate function and return the result as a simple list    
univ_school = sum(univ_school,[])
univ_rank = sum(univ_rank,[])
univ_location = sum(univ_location,[])

#Concaatenate the data into a single dataframe
school_rank_location = pd.DataFrame([univ_school,univ_rank,univ_location], index=['School Name','Rank','Location'])
school_rank_location = school_rank_location.T
 


In [3]:
#Extract the Cost, Facility, Faculty, Programme, Student and Recruiter data from The Economist 
# Ranking, Application and Accreditation details are not required for our analysis

root_web = 'http://www.economist.com/'
university_school_details = []
for  j in range(100):
    for k in range(9):
        if (k != 0 and k != 2 and k != 8):
            school_full_url = root_web + univ_school_url[j] + '?tab=' + str(k)
            response = opener.open(school_full_url)
            page = response.read()
            #print 'j=',j
            soup = BeautifulSoup(page)
            university_school_details.append(BeautifulSoup(page))

In [131]:
univ_facilities_dict = {}; univ_cost_dict = {}; univ_faculty_dict = {}; univ_programme_dict = {}; 
univ_students_dict = {}; univ_recruiters_dict = {}
#i = 0
for i in range(600):  # replace range 600
    school_name = university_school_details[i].find_all('h2')
    school_name = school_name[1].get_text().encode('utf-8')
    if (i % 6 == 0 ):
        j = 0
    else:
        j = j + 1
    
    if (j == 0):
        univ_cost = university_school_details[i].find_all('tr', {'class':['odd', 'even']} )
        univ_cost_dict[school_name] = univ_cost
    if (j == 1):
        univ_facilities = university_school_details[i].find_all('tr', {'class':['odd', 'even']} )
        univ_facilities_dict[school_name] = univ_facilities
    if (j == 2):
        univ_faculty = university_school_details[i].find_all('tr', {'class':['odd', 'even']} )
        univ_faculty_dict[school_name] = univ_faculty
    if (j == 3):
        univ_programme = university_school_details[i].find_all('tr', {'class':['odd', 'even']} )
        univ_programme_dict[school_name] = univ_programme
    if (j == 4):
        univ_students = university_school_details[i].find_all('tr', {'class':['odd', 'even']} )
        univ_students_dict[school_name] = univ_students
    if (j == 5):
        univ_recruiters = university_school_details[i].find_all('tr', {'class':['odd', 'even']} )
        univ_recruiters_dict[school_name] = univ_recruiters

In [154]:
#print univ_cost_dict     

In [132]:

str_univ_cost = ''
 
for i in univ_cost_dict.keys():
    str_univ_cost = str(univ_cost_dict[i])
    univ_cost_dict[i] = str_univ_cost.split('</tr>,')[2:]

#print univ_cost_dict.items()

for i in univ_cost_dict.keys():
    univ_cost_details = {}; variable = '' ; value = ''; val = []
    val = univ_cost_dict[i]
    for j in range(len(val)):
        val = univ_cost_dict[i]
        variable, value = re.findall('<td>.*?</td>'  ,val[j])
        variable = re.sub('<[^<]+?>', '', variable)
        variable = re.sub( '[\$()' '\/]','' ,variable)
        variable = variable.strip()
        value = re.sub('<[^<]+?>', '', value)
        value = re.sub( '[\$' '\/,]','' ,value)
        value = re.sub( '\(.*?\)','' ,value)
        value = value.strip()
        univ_cost_details[variable] = value
    univ_cost_dict[i] = univ_cost_details   
    


In [133]:
str_univ_faculty = ''
 
for i in univ_faculty_dict.keys():
    str_univ_faculty = str(univ_faculty_dict[i])
    univ_faculty_dict[i] = str_univ_faculty.split('</tr>,')[2:]

#print univ_faculty_dict.items()

for i in univ_faculty_dict.keys():
    univ_faculty_details = {}; variable = '' ; value = ''; val = []
    val = univ_faculty_dict[i]
    for j in range(len(val)):
        val = univ_faculty_dict[i]
        variable, value = re.findall('<td>.*?</td>'  ,val[j])
        variable = re.sub('<[^<]+?>', '', variable)
        variable = re.sub( '[\$()' '\/]','' ,variable)
        variable = variable.strip()
        value = re.sub('<[^<]+?>', '', value)
        value = re.sub( '[\$' '\/,]','' ,value)
        value = re.sub('\(.*?\)' ,'' ,value)
        value = value.strip()
        univ_faculty_details[variable] = value
    univ_faculty_dict[i] = univ_faculty_details   

In [155]:
#print univ_faculty_dict.items()

    


In [134]:
str_univ_facilities = ''
 
for i in univ_facilities_dict.keys():
    str_univ_facilities = str(univ_facilities_dict[i])
    univ_facilities_dict[i] = str_univ_facilities.split('</tr>,')[2:]

#print univ_facilities_dict.items()

for i in univ_facilities_dict.keys():
    univ_facilities_details = {}; variable = '' ; value = ''; val = []
    val = univ_facilities_dict[i]
    for j in range(len(val)):
        val = univ_facilities_dict[i]
        variable, value = re.findall('<td>.*?</td>'  ,val[j])
        variable = re.sub('<[^<]+?>', '', variable)
        variable = re.sub( '[\$()' '\/]','' ,variable)
        variable = variable.strip()
        value = re.sub('<[^<]+?>', '', value)
        value = re.sub( '[\$' '\/,]','' ,value)
        value = re.sub( '\(.*?\)','' ,value)
        value = value.strip()
        univ_facilities_details[variable] = value
    univ_facilities_dict[i] = univ_facilities_details   
    


In [108]:
#print univ_facilities_dict.items()

[('University of Chicago \xe2\x80\x93 Booth School of Business', {'Student assessment of facilities out of 5': '4.67'})]


In [135]:
str_univ_programme = ''
 
for i in univ_programme_dict.keys():
    str_univ_programme = str(univ_programme_dict[i])
    univ_programme_dict[i] = str_univ_programme.split('</tr>,')[2:]


for i in univ_programme_dict.keys():
    univ_programme_details = {}; variable = '' ; value = ''; val = []
    val = univ_programme_dict[i]
    for j in range(len(val)):
        val = univ_programme_dict[i]
        variable, value = re.findall('<td>.*?</td>'  ,val[j])
        variable = re.sub('<[^<]+?>', '', variable)
        variable = re.sub( '[\$()' '\/]','' ,variable)
        variable = variable.strip()
        value = re.sub('<[^<]+?>', '', value)
        value = re.sub( '[\$' '\/,]','' ,value)
        value = re.sub('\(.*?\)',''  , value)
        value = value.strip()
        univ_programme_details[variable] = value
    univ_programme_dict[i] = univ_programme_details  
    
#print univ_programme_dict.items()

In [136]:
str_univ_students = ''
 
for i in univ_students_dict.keys():
    str_univ_students = str(univ_students_dict[i])
    univ_students_dict[i] = str_univ_students.split('</tr>,')[2:]


for i in univ_students_dict.keys():
    univ_students_details = {}; variable = '' ; value = ''; val = []
    val = univ_students_dict[i]
    for j in range(len(val)):
        val = univ_students_dict[i]
        variable, value = re.findall('<td>.*?</td>'  ,val[j])
        variable = re.sub('<[^<]+?>', '', variable)
        variable = re.sub( '[\$()' '\/\']','' ,variable)
        #variable = re.sub( '[\']','' ,variable)
        variable = variable.strip()
        value = re.sub('<[^<]+?>', '', value)
        value = re.sub( '[\$' '\/]','' ,value)
        value = re.sub( '\(.*?\)','' ,value)
        value = value.strip()
        univ_students_details[variable] = value
    univ_students_dict[i] = univ_students_details  
    
#print univ_students_dict.items()

In [137]:
str_univ_recruiters = ''
 
for i in univ_recruiters_dict.keys():
    str_univ_recruiters = str(univ_recruiters_dict[i])
    univ_recruiters_dict[i] = str_univ_recruiters.split('</tr>,')[2:]


for i in univ_recruiters_dict.keys():
    univ_recruiters_details = {}; variable = '' ; value = ''; val = []
    val = univ_recruiters_dict[i]
    for j in range(len(val)):
        val = univ_recruiters_dict[i]
        variable, value = re.findall('<td>.*?</td>'  ,val[j])
        variable = re.sub('<[^<]+?>', '', variable)
        variable = re.sub( '[\$()' '\/\']','' ,variable)
        #variable = re.sub( '[\']','' ,variable)
        variable = variable.strip()
        value = re.sub('<[^<]+?>', '', value)
        value = re.sub( '[\$' '\/]','' ,value)
        value = re.sub( '\(.*?\)','' ,value)
        value = value.strip()
        univ_recruiters_details[variable] = value
    univ_recruiters_dict[i] = univ_recruiters_details  
    
#print univ_recruiters_dict.items()

In [138]:
import pandas as pd
df_faculty = pd.DataFrame(univ_faculty_dict.values())
df_faculty_school = pd.DataFrame(univ_faculty_dict.keys(), columns = ['School Name'])
df_faculty_info = pd.concat([df_faculty_school,df_faculty], axis = 1 )
#df_faculty_info

In [139]:

df_recruiters = pd.DataFrame(univ_recruiters_dict.values())
df_recruit_school = pd.DataFrame(univ_recruiters_dict.keys(), columns = ['School Name'])
df_recruitment_info = pd.concat([df_recruit_school,df_recruiters], axis = 1 )
#df_recruitment_info

In [140]:
df_students = pd.DataFrame(univ_students_dict.values())
df_students_school = pd.DataFrame(univ_students_dict.keys(), columns = ['School Name'])
df_students_info = pd.concat([df_students_school,df_students], axis = 1 )
#df_students_info

In [156]:
df_cost = pd.DataFrame(univ_cost_dict.values())
df_cost_school = pd.DataFrame(univ_cost_dict.keys(), columns = ['School Name'])
df_cost_info = pd.concat([df_cost_school,df_cost], axis = 1 )
#df_cost_info

In [142]:
df_facilities = pd.DataFrame(univ_facilities_dict.values())
df_facilities_school = pd.DataFrame(univ_facilities_dict.keys(), columns = ['School Name'])
df_facilities_info = pd.concat([df_facilities_school,df_facilities], axis = 1 )
#df_facilities_info

In [143]:
df_programme = pd.DataFrame(univ_programme_dict.values())
df_programme_school = pd.DataFrame(univ_programme_dict.keys(), columns = ['School Name'])
df_programme_info = pd.concat([df_programme_school,df_programme], axis = 1 )
#df_programme_info

In [144]:
school_info = [df_cost_info, df_facilities_info, df_faculty_info, df_programme_info ,df_recruitment_info, df_students_info  ]
school_merged_info = reduce(lambda left, right: pd.merge(left, right, on ="School Name" ), school_info )

In [119]:
school_merged_info.head()

,School Name,"Accommodation costs off campus, per year,","Accommodation costs on campus, per year,",Application fees,Comments,Criteria on which aid is granted,Financial aid available,Programme fees,Type of aid available,Student assessment of facilities out of 5,...,Average GMAT score,Average age,Average number of years work experience,Europe,Gender diversity score out of 100,Geographical diversity score out of 100,North America,Number of applicants per full-time place,Other Americas,Student rating of culture and classmates out of 5
0,University of Chicago – Booth School of Business,14520,na,250,Per year next academic year,Merit,Yes,63980,,4.67,...,730,27,4.6,10,72.0,52.9,58,7,12,4.60


In [145]:
column_names = ['School Name','Programme fees', 'Student rating of faculty out of 5', 'Ratio of faculty to students', 'Student rating of programme out of 5', 'Average GMAT score',
'Average number of years work experience', 'Average age', 'Student rating of culture and classmates out of 5', 'AfricaMiddle East', 'Other Americas', 'North America', 'Europe', 'AsiaAustralasia', 'Percentage who received a job offer within three months of graduation', 'Post-MBA salary', 'Increase on pre-MBA salary %',
'Principal recruiters of graduates', 'Student rating of careers service out of 5','Percentage of graduates finding jobs through careers services','Geographical diversity score out of 100'
]
school_info_final = school_merged_info[column_names]
columns_for_analysis = ['School Name','Programme fees', 'Student rating of faculty out of 5', 'Ratio of faculty to students', 'Student rating of programme out of 5', 'Average GMAT score',
'Average number of years work experience', 'Average age', 'Student rating of culture and classmates out of 5', 'Percentage who received a job offer within three months of graduation', 'Post-MBA salary', 'Increase on pre-MBA salary %','Principal recruiters of graduates', 'Student rating of careers service out of 5','Percentage of graduates finding jobs through careers services','Geographical diversity score out of 100']
school_info_for_analysis = school_info_final[columns_for_analysis]


In [146]:
full_school_data = pd.merge(school_rank_location, school_info_for_analysis, on ='School Name', how='inner')
#full_school_data

In [147]:
#full_school_data
full_school_data['Average number of years work experience'].replace(',', '.', inplace = True, regex = True)
full_school_data['Principal recruiters of graduates'].replace('&amp;', '&', inplace = True, regex = True)
full_school_data['Student rating of culture and classmates out of 5'].replace(',', '.', inplace = True, regex = True)
full_school_data['Post-MBA salary'].replace(',', '', inplace = True, regex = True)

In [148]:
full_school_data.shape

(100, 18)

In [152]:
#full_school_data

In [150]:
full_school_data.to_csv( 'full_school_data.csv' , index = False)

In [129]:
dfs = pd.read_csv('full_school_data.csv',encoding ='utf-8')